In [1]:
import torch
import os
from torch_geometric.data import Data
from torch_geometric.data import InMemoryDataset
import torch_geometric.utils
import torch.nn.functional as F
from torch import Tensor
from typing import Tuple
from typing import List

# Define the paths to each file
base_path = r'C:\Users\firda\Downloads\QM9\QM9' 
adjacency_file = os.path.join(base_path, 'QM9_A.txt')
graph_indicator_file = os.path.join(base_path, 'QM9_graph_indicator.txt')
node_attributes_file = os.path.join(base_path, 'QM9_node_attributes.txt')  

# Load adjacency list
edges = []
with open(adjacency_file, 'r') as f:
    for line in f:
        src, dst = map(int, line.strip().split(','))
        edges.append((src - 1, dst - 1))  # Subtract 1 to start node indexing from 0

# Load graph indicators
graph_indicator = []
with open(graph_indicator_file, 'r') as f:
    for line in f:
        graph_indicator.append(int(line.strip()) - 1)

# Load node features
node_features = []
with open(node_attributes_file, 'r') as f:
    for line in f:
        features = list(map(float, line.strip().split(',')))
        node_features.append(features)

# Convert lists to tensors
edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
node_features = torch.tensor(node_features, dtype=torch.float)
graph_indicator = torch.tensor(graph_indicator, dtype=torch.long)
#print(edge_index)
#print(graph_indicator)
#print(node_features)

In [2]:
# Define the number of graphs to process since they are much more than 1000
num_graphs_to_process = 1000
data_list = []  # List to store Data objects for each graph

for g_id in range(num_graphs_to_process):
    # Identify nodes belonging to the current graph
    graph_nodes = torch.where(graph_indicator == g_id)[0]
    
    # Select node features for this graph
    x = node_features[graph_nodes]
    
    # Filter edges for this graph based on the graph_indicator
    mask = (graph_indicator[edge_index[0]] == g_id) & (graph_indicator[edge_index[1]] == g_id)
    edge_index_graph = edge_index[:, mask]
    
    # Reindex the edges for this graph
    node_map = {old_idx: new_idx for new_idx, old_idx in enumerate(graph_nodes.tolist())}
    edge_index_graph = torch.tensor([
        [node_map[i.item()] for i in edge_index_graph[0]],
        [node_map[i.item()] for i in edge_index_graph[1]]
    ], dtype=torch.long)
    
    # Create the Data object for the current graph and add it to the list
    data = Data(x=x, edge_index=edge_index_graph)
    data_list.append(data)
print(data_list)

[Data(x=[5, 16], edge_index=[2, 8]), Data(x=[4, 16], edge_index=[2, 6]), Data(x=[3, 16], edge_index=[2, 4]), Data(x=[4, 16], edge_index=[2, 6]), Data(x=[3, 16], edge_index=[2, 4]), Data(x=[4, 16], edge_index=[2, 6]), Data(x=[8, 16], edge_index=[2, 14]), Data(x=[6, 16], edge_index=[2, 10]), Data(x=[7, 16], edge_index=[2, 12]), Data(x=[6, 16], edge_index=[2, 10]), Data(x=[7, 16], edge_index=[2, 12]), Data(x=[6, 16], edge_index=[2, 10]), Data(x=[11, 16], edge_index=[2, 20]), Data(x=[9, 16], edge_index=[2, 16]), Data(x=[9, 16], edge_index=[2, 16]), Data(x=[9, 16], edge_index=[2, 18]), Data(x=[7, 16], edge_index=[2, 14]), Data(x=[10, 16], edge_index=[2, 18]), Data(x=[9, 16], edge_index=[2, 16]), Data(x=[8, 16], edge_index=[2, 14]), Data(x=[14, 16], edge_index=[2, 26]), Data(x=[12, 16], edge_index=[2, 22]), Data(x=[6, 16], edge_index=[2, 10]), Data(x=[5, 16], edge_index=[2, 8]), Data(x=[4, 16], edge_index=[2, 6]), Data(x=[6, 16], edge_index=[2, 10]), Data(x=[5, 16], edge_index=[2, 8]), Data(

In [3]:
def create_actionable_mask(feature_matrix: torch.Tensor, non_actionable_indices: List[int]) -> torch.Tensor:
    """
    Creates a binary actionable mask for a given feature matrix.

    Args:
        feature_matrix (torch.Tensor): Node feature matrix (num_nodes, num_features).
        non_actionable_indices (List[int]): List of indices for non-actionable features.

    Returns:
        torch.Tensor: Binary mask (1 for actionable, 0 for non-actionable) with shape (num_features,).
    """
    # Get the number of features from the feature matrix
    num_features = feature_matrix.size(1)

    # Initialize the mask as all ones (all features are actionable by default)
    actionable_mask = torch.ones(num_features)

    # Set non-actionable indices to 0
    for idx in non_actionable_indices:
        if idx < num_features:  # Ensure the index is within bounds
            actionable_mask[idx] = 0

    return actionable_mask

In [4]:
maschera = create_actionable_mask(data_list[3].x, [0,13])
print(maschera)

tensor([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.])
